In [37]:
import os

In [38]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml\\AI-powered-Bank-Product-Recommender-Chatbot'

In [39]:
os.chdir("../.")

In [40]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml'

In [41]:
project_dir = "C:/Users/RICH-FILES/Desktop/ml/AI-powered-Bank-Product-Recommender-Chatbot"
os.chdir(project_dir)

In [42]:
from dataclasses import dataclass
from pathlib import Path


    
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    raw_data_dir: Path
    data_file: Path
    customers_csv: Path
    products_csv: Path
    customers_table: str
    products_table: str

    

In [43]:
from BankProducts.constants import *
from BankProducts.utils.common import read_yaml, create_directories

In [44]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,    
        params_filepath = PARAMS_FILE_PATH,
        #schema_filepath: str = SCHEMA_FILE_PATH,
        ):
       
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        #self.schema = read_yaml(schema_filepath)        
    
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        table_config = self.config.data_generation.table
    
        create_directories([self.config.artifacts_root])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            raw_data_dir=Path(config.raw_data_dir),
            customers_table=table_config.customers,
            products_table=table_config.products,
            data_file=Path(config.data_file),
            customers_csv=Path(config.customers_csv),
            products_csv=Path(config.products_csv)
            
        )
        
        return data_ingestion_config

In [45]:
import os
import logging
import sqlite3
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
from BankProducts import logger

In [46]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def extract_and_save_data(self):
        """Extract customers and products data from SQLite DB and save as CSV."""
        logging.info(f"Connecting to database: {self.config.data_file}")
        conn = sqlite3.connect(self.config.data_file)

        try:
            customers_df = pd.read_sql_query(f"SELECT * FROM {self.config.customers_table}", conn)
            products_df = pd.read_sql_query(f"SELECT * FROM {self.config.products_table}", conn)
            logging.info(f"Successfully read tables {self.config.customers_table} and {self.config.products_table}")
        finally:
            conn.close()
            logging.info("Database connection closed.")

        # Ensure directories exist
        Path(self.config.raw_data_dir).mkdir(parents=True, exist_ok=True)

        # Save to CSV at correct file paths
        customers_df.to_csv(self.config.customers_csv, index=False)
        products_df.to_csv(self.config.products_csv, index=False)
        logging.info(f"Customers data saved to: {self.config.customers_csv}")
        logging.info(f"Products data saved to: {self.config.products_csv}")

        return customers_df, products_df


In [47]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.extract_and_save_data()
except Exception as e:
    logger.exception(e)
    raise e


[2025-05-24 14:56:25,527: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-24 14:56:25,548: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-24 14:56:25,564: INFO: common: created directory at: artifacts]
[2025-05-24 14:56:25,566: INFO: common: created directory at: artifacts]
[2025-05-24 14:56:25,567: INFO: 4158832910: Connecting to database: artifacts\data_generation\bank_products_recommender.db]
[2025-05-24 14:56:25,984: INFO: 4158832910: Successfully read tables customers and products]
[2025-05-24 14:56:25,987: INFO: 4158832910: Database connection closed.]
[2025-05-24 14:56:26,256: INFO: 4158832910: Customers data saved to: artifacts\data_ingestion\raw_data\bank_customers.csv]
[2025-05-24 14:56:26,257: INFO: 4158832910: Products data saved to: artifacts\data_ingestion\raw_data\product_catalog.csv]
